<a href="https://colab.research.google.com/github/arandi-ar/feature_engineering-/blob/main/Basic_Feature_Engineering_in_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sklearn

In [ ]:
!wget https://raw.githubusercontent.com/gstripling/training-data-analyst/master/housing_pre-proc.csv

--2021-10-27 13:24:28--  https://raw.githubusercontent.com/gstripling/training-data-analyst/master/housing_pre-proc.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1435069 (1.4M) [text/plain]
Saving to: ‘housing_pre-proc.csv’

housing_pre-proc.cs 100%[===================>]   1.37M  --.-KB/s    in 0.03s   

2021-10-27 13:24:28 (39.3 MB/s) - ‘housing_pre-proc.csv’ saved [1435069/1435069]



In [ ]:
import os
import tensorflow.keras

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

from tensorflow import feature_column as fc
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

print(tf.version.VERSION)

2.6.0


In [ ]:
housing_df = pd.read_csv('housing_pre-proc.csv', error_bad_lines=False)
housing_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [ ]:
housing_df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


# Partiendo el dataset para ML
Tenemos que partirlo el CSV en train, validation y test

In [ ]:
train, test = train_test_split(housing_df, test_size=0.2)
train, val = train_test_split(housing_df, test_size=0.2)

Vamos a guardarlos en archivos diferentes

In [ ]:
train.to_csv('housing-train.csv', encoding='utf-8', index=False)
val.to_csv('housing-val.csv', encoding='utf-8', index=False)
test.to_csv('housing-test.csv', encoding='utf-8', index=False)

In [ ]:
!ls -la

total 3084
drwxr-xr-x 1 root root    4096 Oct 26 13:55 .
drwxr-xr-x 1 root root    4096 Oct 26 13:42 ..
drwxr-xr-x 4 root root    4096 Oct  8 13:44 .config
-rw-r--r-- 1 root root 1435069 Oct 26 13:49 housing_pre-proc.csv
-rw-r--r-- 1 root root  282717 Oct 26 13:55 housing-test.csv
-rw-r--r-- 1 root root 1129914 Oct 26 13:55 housing-train.csv
-rw-r--r-- 1 root root  282634 Oct 26 13:55 housing-val.csv
drwxr-xr-x 1 root root    4096 Oct  8 13:45 sample_data


# Lab1: crear un input pipeline usando tf.data

A continuacion vamos a encapsular el dataframe con tf.data. Esto nos va a permitir usar feature columns como un puente para mapear las columnas en el dataframe de pandas con los features usados para entrenar el modelo

In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('median_house_value')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

A continuacipon inicializamos los dataset de entrenamiento y validacion

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)

In [ ]:
# vamos a ver que nos trae

for feature_batch, label_batch in train_ds.take(1):
  print('todas las features', list(feature_batch.keys()))
  print('Un batch de households', feature_batch['households'])
  print('Un batch de ocean_proximity', feature_batch['ocean_proximity'])
  print('A batch of targets', label_batch)

todas las features ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'ocean_proximity']
Un batch de households tf.Tensor(
[ 322.  430.  234.  395.  342.  559.  557.  321.  707.  351. 1790.  642.
  913.  388.  398.  960.  497.  427.  317.  266.  351.   22.  592.  267.
  673.  343.  620.  268.  425.  719.  395.  464.], shape=(32,), dtype=float64)
Un batch de ocean_proximity tf.Tensor(
[b'NEAR BAY' b'<1H OCEAN' b'<1H OCEAN' b'<1H OCEAN' b'<1H OCEAN'
 b'NEAR OCEAN' b'<1H OCEAN' b'INLAND' b'<1H OCEAN' b'INLAND' b'INLAND'
 b'INLAND' b'<1H OCEAN' b'INLAND' b'INLAND' b'<1H OCEAN' b'<1H OCEAN'
 b'INLAND' b'NEAR OCEAN' b'<1H OCEAN' b'NEAR OCEAN' b'NEAR BAY'
 b'NEAR OCEAN' b'INLAND' b'INLAND' b'INLAND' b'INLAND' b'NEAR OCEAN'
 b'INLAND' b'INLAND' b'INLAND' b'INLAND'], shape=(32,), dtype=string)
A batch of targets tf.Tensor(
[142600. 260500. 127400. 500001. 350000. 185600. 225000.  98100. 202600.
  70900. 145500.  65000. 2

## Columnas numericas
La salida de una feature column se convierte en la entrada del modelo

In [ ]:
numeric_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
                'total_bedrooms', 'population', 'households', 'median_income']

## Scaler function
Es muy importante que todas las variables numericas este normalizadas antes de subirla

In [ ]:
def get_scal(feature):
  def minmax(x):
    mini = train[feature].min()
    maxi = train[feature].max()
    return (x - mini)/(maxi-mini)
    return(minmax)

In [ ]:
feature_columns = []
for header in numeric_cols:
  scal_input_fn = get_scal(header)
  feature_columns.append(fc.numeric_cols(header,
                                         normalizer_fn=scal_input_fn))

NameError: ignored